In [1]:
import timm
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from functools import partial

encoder = timm.create_model('vit_small_patch14_dinov2', pretrained=True, dynamic_img_size=True)

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torchvision.transforms.v2 as v2
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import numpy as np

class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image):
        image2 = torch.clone(image)
        if len(image2.shape) == 4:
            # batched
            image2 = image2.permute(1, 0, 2, 3)
        for t, m, s in zip(image2, self.mean, self.std):
            t.mul_(s).add_(m)
        return image2.permute(1, 0, 2, 3)
    
norm = v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
unnorm = UnNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

class TorchPCA(object):

    def __init__(self, n_components):
        self.n_components = n_components

    def fit(self, X):
        self.mean_ = X.mean(dim=0)
        unbiased = X - self.mean_.unsqueeze(0)
        U, S, V = torch.pca_lowrank(unbiased, q=self.n_components, center=False, niter=4)
        self.components_ = V.T
        self.singular_values_ = S
        return self

    def transform(self, X):
        t0 = X - self.mean_.unsqueeze(0)
        projected = t0 @ self.components_.T
        return projected


def pca(image_feats_list, dim=3, fit_pca=None, max_samples=None):
    device = image_feats_list[0].device

    def flatten(tensor, target_size=None):
        if target_size is not None and fit_pca is None:
            tensor = F.interpolate(tensor, (target_size, target_size), mode="bilinear")
        B, C, H, W = tensor.shape
        return tensor.permute(1, 0, 2, 3).reshape(C, B * H * W).permute(1, 0).detach().cpu()

    if len(image_feats_list) > 1 and fit_pca is None:
        target_size = image_feats_list[0].shape[2]
    else:
        target_size = None

    flattened_feats = []
    for feats in image_feats_list:
        flattened_feats.append(flatten(feats, target_size))
    x = torch.cat(flattened_feats, dim=0)

    # Subsample the data if max_samples is set and the number of samples exceeds max_samples
    if max_samples is not None and x.shape[0] > max_samples:
        indices = torch.randperm(x.shape[0])[:max_samples]
        x = x[indices]

    if fit_pca is None:
        fit_pca = TorchPCA(n_components=dim).fit(x)

    reduced_feats = []
    for feats in image_feats_list:
        x_red = fit_pca.transform(flatten(feats))
        if isinstance(x_red, np.ndarray):
            x_red = torch.from_numpy(x_red)
        x_red -= x_red.min(dim=0, keepdim=True).values
        x_red /= x_red.max(dim=0, keepdim=True).values
        B, C, H, W = feats.shape
        reduced_feats.append(x_red.reshape(B, H, W, dim).permute(0, 3, 1, 2).to(device))

    return reduced_feats, fit_pca

from pytorch_lightning import seed_everything

def _remove_axes(ax):
    ax.xaxis.set_major_formatter(plt.NullFormatter())
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    ax.set_xticks([])
    ax.set_yticks([])


def remove_axes(axes):
    if len(axes.shape) == 2:
        for ax1 in axes:
            for ax in ax1:
                _remove_axes(ax)
    else:
        for ax in axes:
            _remove_axes(ax)

In [3]:
#from dl_toolbox.datasets import Rellis3d
#from torchvision import tv_tensors
#
#tf = v2.Compose([
#    v2.RandomCrop(size=(672, 672)),
#    v2.ToDtype(
#        dtype={tv_tensors.Image: torch.float32, tv_tensors.Mask: torch.int64, "others":None}, 
#        scale=True
#    ),
#    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#])
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
#rellis = '/data/Rellis-3D'
#imgs = [rellis+'/00000/pylon_camera_node/frame000000-1581624652_750.jpg']
#msks = [rellis+'/00000/pylon_camera_node_label_id/frame000000-1581624652_750.png']
#dataset = Rellis3d(
#    imgs=imgs,
#    msks=msks,
#    merge='all19',
#    transforms=tf
#)
#elem = dataset[0]
#image, mask = elem['image'].to(device).unsqueeze(0), elem['label']
#h = 672 // 14
#w = 672 // 14
#encoder.to(device)
#lr_feats = encoder.forward_features(image)
#lr_feats = lr_feats[:,encoder.num_prefix_tokens:,...]
#lr_feats = lr_feats.reshape(-1, h, w, 384).permute(0,3,1,2).detach().cpu()
#hr_feats_bili = v2.functional.resize(lr_feats, (672, 672), Image.BILINEAR)

In [4]:
#from concurrent.futures import ProcessPoolExecutor
#
#ref = image[0].permute(1,2,0).to('cpu').numpy()
#source_upsampled = hr_feats_bili[0].permute(1,2,0).to('cpu').numpy()
#
#NB_CHANNELS = 384
#
#scale = 48 / 672
#radius = 2
#diameter = 2 * radius + 1
#step = int(np.ceil(1 / scale))
#padding = radius * step
#sigma_spatial = 2.5
#sigma_range = np.std(ref)
#
#reference = np.pad(ref, ((padding, padding), (padding, padding), (0, 0)), 'symmetric').astype(np.float32)
#source_upsampled = np.pad(source_upsampled, ((padding, padding), (padding, padding), (0, 0)), 'symmetric').astype(np.float32)
#
## Spatial Gaussian function.
#x, y = np.meshgrid(np.arange(diameter) - radius, np.arange(diameter) - radius)
#kernel_spatial = np.exp(-1.0 * (x**2 + y**2) /  (2 * sigma_spatial**2))
##kernel_spatial = np.repeat(kernel_spatial, 3).reshape(-1, 3)
#kernel_spatial = np.reshape(-1,1)
#
## Lookup table for range kernel.
#lut_range = np.exp(-1.0 * np.arange(256)**2 / (2 * sigma_range**2))
#
#def process_row(y):
#    result = np.zeros((ref.shape[1], NB_CHANNELS))
#    y += padding
#    for x in range(padding, reference.shape[1] - padding):
#        I_p = reference[y, x]
#        patch_reference = reference[y - padding:y + padding + 1:step, x - padding:x + padding + 1:step].reshape(-1, 3)
#        patch_source_upsampled = source_upsampled[y - padding:y + padding + 1:step, x - padding:x + padding + 1:step].reshape(-1, NB_CHANNELS)
#
#        kernel_range = lut_range[np.abs(patch_reference - I_p).astype(int)]
#        norm = np.linalg.norm(patch_reference-I_p, axis=1, keepdims=True)
#        kernel_range = np.exp(-1.0 * norm**2 / (2 * sigma_range**2))
#        weight = kernel_range * kernel_spatial
#        prod = weight * patch_source_upsampled
#        k_p = weight.sum(axis=0)
#        res = np.round(np.sum(weight * patch_source_upsampled, axis=0) / k_p)
#        result[x - padding] = res
#    return result
#
#executor = ProcessPoolExecutor()
#result = executor.map(process_row, range(ref.shape[0]))
#executor.shutdown(True)
#hr_feats = torch.Tensor(np.array(list(result))) 
#print(hr_feats.shape)

In [5]:
#[lr_feats_pca], _ = pca([lr_feats])
#[hr_feats_bili_pca], _ = pca([hr_feats_bili])
#[hr_feats_jbu_pca], _ = pca([hr_feats.permute(2,0,1).unsqueeze(0)])
#
#fig, ax = plt.subplots(2, 2, figsize=(15, 15))
#ax[0,0].imshow(v2.functional.to_pil_image(unnorm(image).squeeze(0)))
#ax[0,0].set_title("Ref Image")
#ax[0,1].imshow(v2.functional.to_pil_image(lr_feats_pca[0]))
#ax[0,1].set_title("low res features")
#ax[1,0].imshow(v2.functional.to_pil_image(hr_feats_bili_pca[0]))
#ax[1,0].set_title("Bilinear upsampled features")
#ax[1,1].imshow(v2.functional.to_pil_image(hr_feats_jbu_pca[0]))
#ax[1,1].set_title("JBU features")
#remove_axes(ax)
#plt.show()

In [6]:
from timm.models.layers import trunc_normal_

def init_weights(m):
    if isinstance(m, nn.Linear):
        trunc_normal_(m.weight, std=0.02)
        if isinstance(m, nn.Linear) and m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
        nn.init.constant_(m.bias, 0)
        nn.init.constant_(m.weight, 1.0)

class DecoderLinear(nn.Module):
    def __init__(self, n_cls, patch_size, d_encoder):
        super().__init__()

        self.d_encoder = d_encoder
        self.patch_size = patch_size
        self.n_cls = n_cls

        self.head = nn.Linear(self.d_encoder, n_cls)
        self.apply(init_weights)

    @torch.jit.ignore
    def no_weight_decay(self):
        return set()

    def forward(self, x, im_size):
        H, W = im_size
        GS = H // self.patch_size
        x = self.head(x)
        x = rearrange(x, "b (h w) c -> b c h w", h=GS)

        return x
    
import torch
import torch.nn as nn
import torch.nn.functional as F
import minlora 
import torchmetrics as M
import pytorch_lightning as pl
from dl_toolbox import losses

def get_lora_config(rank):
    return {  # specify which layers to add lora to, by default only add to linear layers
        nn.Linear: {
            "weight": partial(minlora.LoRAParametrization.from_linear, rank=rank),
        },
    }

class Segmenter(pl.LightningModule):
    def __init__(
        self,
        encoder,
        decoder,
        n_cls,
        freeze,
        lora,
        rank
    ):
        super().__init__()
        self.n_cls = n_cls
        self.encoder = encoder
        self.decoder = decoder
        if freeze:
            for param in self.encoder.parameters():
                param.requires_grad = False
        if lora:
            cfg = get_lora_config(rank)
            minlora.add_lora(self.encoder, lora_config=cfg)
        self.acc = M.Accuracy(task='multiclass', num_classes=20)
        self.loss =  losses.CrossEntropy()

    def configure_optimizers(self):
        opt = schedulefree.AdamWScheduleFree(self.parameters(), lr=0.0025)
        return opt
    
    def on_train_epoch_start(self):
        print('\n opt train')
        self.optimizers().train()
        
    def on_validation_start(self):
        print('\n opt eval')
        self.optimizers().eval()  

    @torch.jit.ignore
    def no_weight_decay(self):
        def append_prefix_no_weight_decay(prefix, module):
            return set(map(lambda x: prefix + x, module.no_weight_decay()))

        nwd_params = append_prefix_no_weight_decay("encoder.", self.encoder).union(
            append_prefix_no_weight_decay("decoder.", self.decoder)
        )
        return nwd_params

    def forward(self, im):
        H_ori, W_ori = im.size(2), im.size(3)
        H, W = im.size(2), im.size(3)

        #x = self.encoder(im, return_features=True)
        x = self.encoder.forward_features(im)
        

        # remove CLS/DIST tokens for decoding
        #num_extra_tokens = 1 + self.encoder.distilled
        #x = x[:, num_extra_tokens:]
        x = x[:,self.encoder.num_prefix_tokens:,...]

        masks = self.decoder(x, (H, W))

        masks = F.interpolate(masks, size=(H, W), mode="bilinear")
        #masks = unpadding(masks, (H_ori, W_ori))

        return masks
    
    def training_step(self, batch, batch_idx):
        batch = batch["sup"]
        x = batch["image"]
        y = batch["label"]
        outputs = self.forward(x)
        loss = self.loss(outputs, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x = batch["image"]
        y = batch["label"]
        outputs = self.forward(x)
        loss = self.loss(outputs, y)
        acc = self.acc.update(outputs, y)
        
    def on_validation_epoch_end(self):
        acc = self.acc.compute()
        print("\nAcc: ", acc)
        self.acc.reset()

decoder = DecoderLinear(n_cls=20, d_encoder=encoder.embed_dim, patch_size=14)
model = Segmenter(encoder,decoder,n_cls=20, freeze=True, lora=True, rank=4)

def name_is_lora(name):
    return (
        len(name.split(".")) >= 4
        and (name.split(".")[-4]) == "parametrizations"
        and name.split(".")[-1] in ["lora_A", "lora_B"]
    )

def name_is_decoder(name):
    return (name.split(".")[0]) == "decoder"

def lora_or_decoder(name):
    return name_is_lora(name) or name_is_decoder(name)

def get_params_by_name(model, print_shapes=False, name_filter=None):
    for n, p in model.named_parameters():
        if name_filter is None or name_filter(n):
            if print_shapes:
                print(n, p.shape)
            yield p

print([p[0] for p in list(model.named_parameters())][-10:])
parameters = list(model.parameters())
trainable_parameters = list(get_params_by_name(model, name_filter=lora_or_decoder))
print(
    f"The model will start training with only {sum([int(torch.numel(p)) for p in trainable_parameters])} "
    f"trainable parameters out of {sum([int(torch.numel(p)) for p in parameters])}."
)

['encoder.blocks.11.mlp.fc1.parametrizations.weight.0.lora_B', 'encoder.blocks.11.mlp.fc2.bias', 'encoder.blocks.11.mlp.fc2.parametrizations.weight.original', 'encoder.blocks.11.mlp.fc2.parametrizations.weight.0.lora_A', 'encoder.blocks.11.mlp.fc2.parametrizations.weight.0.lora_B', 'encoder.blocks.11.ls2.gamma', 'encoder.norm.weight', 'encoder.norm.bias', 'decoder.head.weight', 'decoder.head.bias']
The model will start training with only 302612 trainable parameters out of 22358804.


In [ ]:
from dl_toolbox import datamodules
import schedulefree
from dl_toolbox.callbacks import ProgressBar
from torchvision import tv_tensors


tf = v2.Compose([
    v2.RandomCrop(size=(672, 672)),
    v2.ToDtype(
        dtype={tv_tensors.Image: torch.float32, tv_tensors.Mask: torch.int64, "others":None}, 
        scale=True
    ),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dm = datamodules.Rellis3d(
    data_path='/data',
    merge='all19',
    sup=1,
    unsup=5,
    train_tf=tf,
    test_tf=tf,
    batch_size_s=4,
    batch_size_u=4,
    steps_per_epoch=10,
    num_workers=0,
    pin_memory=True
)

trainer = pl.Trainer(
    accelerator='gpu',
    devices=1,
    max_epochs=20,
    limit_train_batches=1,
    limit_val_batches=1,
    callbacks=[ProgressBar()]
)


trainer.fit(
    model,
    datamodule=dm
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
Missing logger folder: /d/pfournie/dl_toolbox/dl_toolbox/à ranger/segmenter/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type               | Params
-----------------------------------------------
0 | encoder | VisionTransformer  | 22.4 M
1 | decoder | DecoderLinear      | 7.7 K 
2 | acc     | MulticlassAccuracy | 0     
3 | loss    | CrossEntropy       | 0     
-----------------------------------------------
302 K     Trainable params
22.1 M    Non-trainable params
22.4 M    Total params
89.435    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]
 opt eval


/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.08it/s]
Acc:  tensor(0.0060, device='cuda:0')
                                                                                                                                                                

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|                                                                                                                            | 0/1 [00:00<?, ?it/s]
 opt train
Epoch 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it, v_num=0]
Validation: 0it [00:00, ?it/s]
 opt eval

Acc:  tensor(0.4648, device='cuda:0')
Epoch 1:   0%|                                                                                                                   | 0/1 [00:00<?, ?it/s, v_num=0]
 opt train
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.23it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
 opt eval

Acc:  tensor(0.6410, device='cuda:0')
Epoch 2:   0%|                                                                                                                   | 0/1 [00:00<?, ?it/s, v_num=0]
 opt train
Ep